# Creating a Basic Feedforward Neural Network for Classification

This notebook demonstrates how to build a basic feedforward neural network (FFNN) for a classification task using Python, TensorFlow, and Keras. We'll use a synthetic dataset for binary classification, preprocess the data, build the model, train it, and evaluate its performance.

**Objectives**:
- Generate a synthetic dataset for classification
- Preprocess the data (scaling)
- Build a feedforward neural network
- Train and evaluate the model
- Visualize the results

## Setup and Imports

Let's import the necessary libraries and set a random seed for reproducibility.

In [7]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

## Generating Synthetic Data

We'll create a synthetic dataset with 1000 samples, 20 features, and 2 classes using `make_classification` from scikit-learn.

In [8]:
# Generate synthetic dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5,
                          n_classes=2, random_state=42)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (800, 20)
Testing set shape: (200, 20)


## Building the Feedforward Neural Network

We'll create a simple FFNN with:
- Input layer (20 features)
- Two hidden layers (64 and 32 neurons, ReLU activation)
- Output layer (1 neuron, sigmoid activation for binary classification)

In [9]:
def build_ffnn(input_shape):
    """Build a feedforward neural network model"""
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Build the model
model = build_ffnn(X_train.shape[1])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

## Training the Model

We'll train the model for 50 epochs with a batch size of 32, using 20% of the training data for validation.

In [10]:
# Train the model
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1)

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.5314 - loss: 0.6941 - val_accuracy: 0.7375 - val_loss: 0.5861
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8042 - loss: 0.5401 - val_accuracy: 0.7812 - val_loss: 0.5039
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8591 - loss: 0.4460 - val_accuracy: 0.8188 - val_loss: 0.4418
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8690 - loss: 0.3739 - val_accuracy: 0.8250 - val_loss: 0.3937
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8769 - loss: 0.3209 - val_accuracy: 0.8438 - val_loss: 0.3556
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8970 - loss: 0.2799 - val_accuracy: 0.8562 - val_loss: 0.3219
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9022 - loss: 0.2473 - val_accuracy: 0.8625 - val_loss: 0.2931
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9343 - loss: 0.2204 - val_accuracy: 0.8562 - val_loss

## Evaluating the Model

Let's evaluate the model's performance on the test set and print the accuracy and classification report.

In [11]:
# Make predictions
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Test Accuracy: 0.9400

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       106
           1       0.94      0.94      0.94        94

    accuracy                           0.94       200
   macro avg       0.94      0.94      0.94       200
weighted avg       0.94      0.94      0.94       200



## Visualizing Training Progress

We'll plot the training and validation loss and accuracy over epochs to visualize the model's learning process.

In [12]:
# Plot training history
plt.figure(figsize=(12, 4))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('training_history.png')
plt.close()

## Explanation of Results

- **Model Architecture**: The FFNN has two hidden layers with ReLU activation for non-linearity and a sigmoid output layer for binary classification.
- **Training**: The model is trained using the Adam optimizer and binary cross-entropy loss, suitable for binary classification.
- **Evaluation**: The classification report provides precision, recall, and F1-score for both classes, while the accuracy gives an overall performance metric.
- **Visualization**: The loss and accuracy plots help identify if the model is overfitting or underfitting.

This basic FFNN can be extended by:
- Adding dropout layers to prevent overfitting
- Tuning hyperparameters (e.g., number of layers, neurons, learning rate)
- Using real-world datasets for specific classification tasks